In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import centrex_TlF as centrex
from fractions import Fraction

from pathlib import Path

In [2]:
path = Path()

data = {}
simulated_heights = {}
for p in path.glob("*.pkl"):
    print(p)
    with open(p, 'rb') as f:
        data[p.stem] = pickle.load(f)
        

Q(1) F1'=1_2 F'=0.pkl
Q(1) F1'=1_2 F'=1.pkl
Q(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=2.pkl
R(1) F1'=5_2 F'=2.pkl
R(1) F1'=5_2 F'=3.pkl


In [3]:
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [4]:
transitions = [
    centrex.transitions.LaserTransition(
        tn[0]+tn[2], 
        F1= convert_to_float(tn.split()[1].split('=')[-1].replace('_', '/')),
        F=int(tn[-1])
    )
    for tn in data.keys()
]
transitions

[Transition(Q(1) F1'=1/2 F'=0),
 Transition(Q(1) F1'=1/2 F'=1),
 Transition(Q(1) F1'=3/2 F'=1),
 Transition(R(1) F1'=3/2 F'=1),
 Transition(R(1) F1'=3/2 F'=2),
 Transition(R(1) F1'=5/2 F'=2),
 Transition(R(1) F1'=5/2 F'=3)]

In [5]:
transitions[0].get_string()

"Q(1) F1'=1/2 F'=0"

In [6]:
df = pd.DataFrame()
for idx, (key, value) in enumerate(data.items()):
    # print("="*52)
    # print(key)
    # print("="*52)
    nphotons_normalized_to_hfs_coupled = 0
    for nphoton, state in zip(value["nphotons"], value["QN"]):
        state = state.find_largest_component()
        if state in transitions[idx].ground_states:
            nphotons_normalized_to_hfs_coupled += nphoton
        # string = f"J={int(state.J)}, F1={state.F1}, F={int(state.F)}, mF={int(state.mF):>2} : {nphoton:.2f}"
    #     print(string)
    # print(f"mean photons scattered from all hfs => {np.mean(value['nphotons']):.2f}")
    # print(f"mean photons scattered from hfs coupled => {nphotons_normalized_to_hfs_coupled / transitions[idx].n_ground:.2f}")
    # print(f"mean photons scattered from branching ratio => {1/(1-np.mean(list(value['branching'].values()))):.2f}")
    other = [
        np.mean(value['nphotons']),
        nphotons_normalized_to_hfs_coupled / transitions[idx].n_ground,
        1/(1-np.mean(list(value['branching'].values()))),
    ]
    df[transitions[idx].get_string()] = value["nphotons"] + other
df.index = [
    f"J={int(state.J)}, F1={state.F1}, F={int(state.F)}, mF={int(state.mF):>2}"
    for state in [s.find_largest_component() for s in value["QN"]] 
    if ((state.J == 1) & (state.electronic_state == 'X'))
] + ["#photons all hfs", "#photons coupled hfs", "#photons branching"]
df

,Q(1) F1'=1/2 F'=0,Q(1) F1'=1/2 F'=1,Q(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=2,R(1) F1'=5/2 F'=2,R(1) F1'=5/2 F'=3
"J=1, F1=0.5, F=0, mF= 0",0.000000e+00,17.963711,4.772385,3.392092,0.000000,0.000000,0.000000
"J=1, F1=0.5, F=1, mF=-1",1.499318e+00,17.831524,2.118363,3.387030,3.473383,1.097652,0.000000
"J=1, F1=0.5, F=1, mF= 0",2.998713e+00,17.704083,2.496730,3.385104,3.473067,1.068219,0.000000
"J=1, F1=0.5, F=1, mF= 1",1.499179e+00,17.831561,2.118389,3.387399,3.473366,1.097826,0.000000
"J=1, F1=1.5, F=1, mF=-1",1.497874e+00,13.636197,4.548719,3.324249,0.007947,2.097951,0.000000
"J=1, F1=1.5, F=1, mF= 0",2.995856e+00,9.742584,4.282889,3.260889,0.007344,2.097103,0.000000
"J=1, F1=1.5, F=1, mF= 1",1.497842e+00,13.637170,4.547879,3.324432,0.007993,2.097903,0.000000
"J=1, F1=1.5, F=2, mF=-2",0.000000e+00,15.752517,2.777799,1.368413,3.472145,2.093791,2.101944
"J=1, F1=1.5, F=2, mF=-1",0.000000e+00,15.942167,2.812923,2.441979,3.470119,2.091472,2.101593
"J=1, F1=1.5, F=2, mF= 0",0.000000e+00,17.889844,4.866058,2.668472,3.471827,2.093027,2.101543


In [7]:
def style_within_percentage(v, props = "", **kwargs):
    value = kwargs.get("value", np.nan)
    percentage = kwargs.get("percentage", 0.1)
    return props if (np.abs(v - value) < percentage*v) else None
    if np.isnan(v):
        return None

In [11]:
percentage_good = 0.1

index_string = "["
for index in df.index[:-3]:
    index_string += '"' + str(index) + '",'
index_string = index_string.strip(",") + "]"

cmd_string = "df.style"
for transition in transitions:
    transition = transition.get_string()
    value = df[transition]["#photons branching"]
    subset_string = '["' +  str(transition) + '"]'
    cmd_string += (
        f".applymap("
        f"style_within_percentage,"
        f"props = 'color:white;background-color:green;opacity: 50%;',"
        f"value = {value},"
        f"percentage = {percentage_good},"
        f'subset = ({index_string}, {subset_string})'
        ')'
    )
df_styled = eval(cmd_string)
df_styled

,Q(1) F1'=1/2 F'=0,Q(1) F1'=1/2 F'=1,Q(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=2,R(1) F1'=5/2 F'=2,R(1) F1'=5/2 F'=3
"J=1, F1=0.5, F=0, mF= 0",0.000000,17.963711,4.772385,3.392092,0.000000,0.000000,0.000000
"J=1, F1=0.5, F=1, mF=-1",1.499318,17.831524,2.118363,3.387030,3.473383,1.097652,0.000000
"J=1, F1=0.5, F=1, mF= 0",2.998713,17.704083,2.496730,3.385104,3.473067,1.068219,0.000000
"J=1, F1=0.5, F=1, mF= 1",1.499179,17.831561,2.118389,3.387399,3.473366,1.097826,0.000000
"J=1, F1=1.5, F=1, mF=-1",1.497874,13.636197,4.548719,3.324249,0.007947,2.097951,0.000000
"J=1, F1=1.5, F=1, mF= 0",2.995856,9.742584,4.282889,3.260889,0.007344,2.097103,0.000000
"J=1, F1=1.5, F=1, mF= 1",1.497842,13.637170,4.547879,3.324432,0.007993,2.097903,0.000000
"J=1, F1=1.5, F=2, mF=-2",0.000000,15.752517,2.777799,1.368413,3.472145,2.093791,2.101944
"J=1, F1=1.5, F=2, mF=-1",0.000000,15.942167,2.812923,2.441979,3.470119,2.091472,2.101593
"J=1, F1=1.5, F=2, mF= 0",0.000000,17.889844,4.866058,2.668472,3.471827,2.093027,2.101543
